In [2]:
import time
from pathlib import Path
from operator import itemgetter

import scanpy as sc
import numpy as np
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from scipy.sparse import csr_matrix
import pandas as pd
from joblib import Parallel, delayed
import os
import torch

import scSLAT
from scSLAT.model import run_SLAT_multi
from scSLAT.viz import build_3D
import scanpy as sc
import numpy as np
import pandas as pd

import scSLAT
from scSLAT.model import Cal_Spatial_Net, load_anndatas, run_SLAT_mlp_AGF, spatial_match, run_SLAT, compute_lisi_for_adata
from scSLAT.viz import match_3D_multi, hist, Sankey
from scSLAT.metrics import region_statistics


In [2]:
import h5py
# 打开 HDF5 文件
file_path_1 = 'D:/ppppaper/data/imputed.h5'
with h5py.File(file_path_1, 'r') as f:
    # 打印文件中所有的组和数据集
    def print_attrs(name, obj):
        print(name)
        for key, val in obj.attrs.items():
            print(f"    {key}: {val}")

    f.visititems(print_attrs)

logcounts


In [5]:
import pyreadr

# 读取RDS文件
result = pyreadr.read_r("D:\ppppaper\data\mRNA.Rds")


In [6]:
# 获取数据
data = result[None]

print(data)

                          uniqueID geneID    pos         x         y  \
0          embryo1_Pos0_cell135_z2  Abcc4   Pos0   670.522   423.266   
1           embryo1_Pos0_cell95_z2  Abcc4   Pos0   458.830  1637.477   
2          embryo1_Pos0_cell326_z2  Abcc4   Pos0  1503.623   436.123   
3          embryo1_Pos0_cell280_z2  Abcc4   Pos0  1273.170  1287.838   
4          embryo1_Pos0_cell168_z2  Abcc4   Pos0   793.611  1096.647   
...                            ...    ...    ...       ...       ...   
12421634    embryo3_Pos39_cell1_z5  Aldh2  Pos39  1104.259  1015.613   
12421635    embryo3_Pos39_cell1_z5  Aldh2  Pos39  1107.656  1010.857   
12421636  embryo3_Pos39_cell199_z5  Aldh2  Pos39  1847.216   798.084   
12421637  embryo3_Pos39_cell158_z5  Aldh2  Pos39  1689.061   850.022   
12421638  embryo3_Pos39_cell185_z5  Aldh2  Pos39  1831.054  1001.953   

             x_global     y_global  x_global_affine  y_global_affine  z  \
0         1627.149503  1335.406858         1.029286        -

In [7]:
print("列名: ", data.columns.tolist())

列名:  ['uniqueID', 'geneID', 'pos', 'x', 'y', 'x_global', 'y_global', 'x_global_affine', 'y_global_affine', 'z', 'seeds', 'intensity']


In [15]:
import pandas as pd

# 读取CSV文件
df = pd.read_csv("D:\ppppaper\SpatialMouseAtlas2020-main\SpatialMouseAtlas2020-main\exprs_norm_embryo2_data.csv")


In [44]:
import pandas as pd
import anndata

# 读取CSV文件，假设第一行是样本名，第一列是基因名
df = pd.read_csv("D:\ppppaper\SpatialMouseAtlas2020-main\SpatialMouseAtlas2020-main\exprs_embryo2_data.csv", index_col=0)

# 将第一行的细胞名设置为列名
gene_names = df.columns

# 将数据转换为 numpy 数组
X_data = df.values

# # 创建AnnData对象
# adata = anndata.AnnData(
#     X=X_data,  # 基因表达矩阵
#     obs=df.columns.to_frame(),  # 观测DataFrame，即细胞名
#     var=df.columns  # 变量，即基因名
# )


# 保存为H5AD文件
# adata.write_h5ad('output.h5ad')
df

,embryo2_Pos29_cell10_z2,embryo2_Pos29_cell10_z5,embryo2_Pos29_cell100_z2,embryo2_Pos29_cell100_z5,embryo2_Pos29_cell101_z2,embryo2_Pos29_cell101_z5,embryo2_Pos29_cell102_z5,embryo2_Pos29_cell103_z2,embryo2_Pos29_cell103_z5,embryo2_Pos29_cell104_z2,...,embryo2_Pos50_cell95_z2,embryo2_Pos50_cell95_z5,embryo2_Pos50_cell96_z2,embryo2_Pos50_cell96_z5,embryo2_Pos50_cell97_z2,embryo2_Pos50_cell97_z5,embryo2_Pos50_cell98_z2,embryo2_Pos50_cell98_z5,embryo2_Pos50_cell99_z2,embryo2_Pos50_cell99_z5
Abcc4,0.000000,0.000000,0.000000,0.769335,0.000000,0.000000,0.000000,0.000000,0.0,1.115657,...,1.341392,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
Acp5,0.733575,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.785119,0.000000,0.000000,0.000000
Acvr1,0.000000,0.000000,0.000000,0.000000,0.000000,0.598444,0.000000,0.000000,0.0,1.115657,...,1.341392,0.000000,0.000000,0.000000,0.000000,0.0,0.785119,0.000000,0.000000,1.408171
Acvr2a,0.000000,0.000000,0.000000,0.769335,1.313838,0.598444,0.389335,0.000000,0.0,0.000000,...,1.341392,0.000000,1.170813,2.347822,0.000000,0.0,0.000000,2.618433,0.000000,1.408171
Adora2b,0.000000,0.000000,1.416037,0.769335,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.836091,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wnt8a,0.000000,0.000000,0.000000,1.268415,0.000000,0.000000,0.389335,0.000000,0.0,0.000000,...,1.341392,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.787428,0.000000
Xist,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
Zfp444,0.733575,2.136394,0.000000,0.000000,1.313838,1.611801,1.349901,0.000000,0.0,0.000000,...,0.000000,1.377628,0.000000,1.606573,1.226207,0.0,1.290701,0.000000,1.293953,1.408171
Zfp57,0.000000,0.000000,0.000000,0.769335,2.448388,0.000000,0.000000,0.000000,0.0,1.115657,...,0.000000,2.069295,0.000000,0.000000,1.226207,0.0,0.000000,0.000000,0.000000,0.000000


In [3]:
adata_1=sc.read_h5ad('D:\ppppaper\data\mRNA.h5ad')
adata_1.var_names_make_unique(join="++")
adata_1

AnnData object with n_obs × n_vars = 12421639 × 12